# Notebook Setup

In [52]:
# Data
import pandas as pd
import numpy as np

# ML/DL
import tensorflow as tf
import tensorflow.keras as k

# Technical
import os
import time
import typing
from IPython.display import clear_output

In [53]:
df = pd.read_csv(r'data\data_clean.csv')
df.drop('Unnamed: 0', axis=1, inplace=True)
df.head()

,label,text
0,0,"jurong point , crazy .. available bugis n grea..."
1,0,ok lar ... joke wif u oni ...
2,1,free entry 2 wkly comp win FA Cup final tkts 2...
3,0,u dun early hor ... u c ...
4,0,"nah I think usf , live"


# Preprocessing

## Tokenization

In [54]:
NUM_WORDS = 5_000
BATCH_SIZE = 64

In [55]:
df['text'] = df['text'].astype(str)

def tokenize(text, tokenizer, num_words):
    tokenizer = tokenizer(num_words=num_words, oov_token='oov')
    tokenizer.fit_on_texts(text)
    return tokenizer.texts_to_sequences(text)

df['token'] = tokenize(df['text'], k.preprocessing.text.Tokenizer, NUM_WORDS)

## Padding

In [56]:
padded = k.preprocessing.sequence.pad_sequences(df['token'], maxlen=None, padding='post')

## Tensors

In [57]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(padded, df['label'], test_size=0.2)
train, val = tf.data.Dataset.from_tensor_slices((x_train, y_train)), tf.data.Dataset.from_tensor_slices((x_val, y_val))
train_batch, val_batch = train.shuffle(len(train)).batch(BATCH_SIZE), val.shuffle(len(val)).batch(BATCH_SIZE)

for text, label in train_batch.take(1):
    print(text, label)

tf.Tensor(
[[3639    1  370 ...    0    0    0]
 [ 101  183 1168 ...    0    0    0]
 [4885  364    3 ...    0    0    0]
 ...
 [  33 1657  323 ...    0    0    0]
 [2777   12   73 ...    0    0    0]
 [   2   19  382 ...    0    0    0]], shape=(64, 78), dtype=int32) tf.Tensor(
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0
 0 0 0 1 0 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0], shape=(64,), dtype=int64)


# Model

In [58]:
import tensorflow.keras.layers as l

vocab_size = NUM_WORDS
model = k.Sequential([
    l.Embedding(vocab_size, 8, input_shape=[padded.shape[1],]),
    l.GlobalAveragePooling1D(),
    l.Dense(8, activation='swish'),
    l.Dropout(0.2),
    l.Dense(1, activation='sigmoid')
])

optimizer = k.optimizers.Adam(learning_rate=0.1)
model.compile(
    optimizer=optimizer,
    loss=k.losses.BinaryCrossentropy(),
    metrics=[k.metrics.BinaryAccuracy(), k.metrics.Recall(), k.metrics.Precision()],
    )

In [59]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 78, 8)             40000     
                                                                 
 global_average_pooling1d_6   (None, 8)                0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_12 (Dense)            (None, 8)                 72        
                                                                 
 dropout_6 (Dropout)         (None, 8)                 0         
                                                                 
 dense_13 (Dense)            (None, 1)                 9         
                                                                 
Total params: 40,081
Trainable params: 40,081
Non-trainable params: 0
__________________________________________________

In [60]:
def scheduler(epoch, lr):
    lr = lr * np.exp(-0.1)
    return lr
cp_lr = tf.keras.callbacks.LearningRateScheduler(scheduler)

hist = model.fit(
    train_batch,
    validation_data=val_batch,
    epochs=100,
    callbacks=[cp_lr]
    )

Epoch 1/100
70/70 [==============================] - 2s 18ms/step - loss: 0.1906 - binary_accuracy: 0.9361 - recall_6: 0.5365 - precision_6: 0.9450 - val_loss: 0.0738 - val_binary_accuracy: 0.9794 - val_recall_6: 0.9123 - val_precision_6: 0.9512 - lr: 0.0905
Epoch 2/100
70/70 [==============================] - 1s 14ms/step - loss: 0.0347 - binary_accuracy: 0.9919 - recall_6: 0.9427 - precision_6: 0.9945 - val_loss: 0.0762 - val_binary_accuracy: 0.9830 - val_recall_6: 0.9064 - val_precision_6: 0.9810 - lr: 0.0819
Epoch 3/100
70/70 [==============================] - 1s 11ms/step - loss: 0.0158 - binary_accuracy: 0.9966 - recall_6: 0.9826 - precision_6: 0.9912 - val_loss: 0.0792 - val_binary_accuracy: 0.9839 - val_recall_6: 0.9240 - val_precision_6: 0.9693 - lr: 0.0741
Epoch 4/100
70/70 [==============================] - 1s 14ms/step - loss: 0.0084 - binary_accuracy: 0.9982 - recall_6: 0.9878 - precision_6: 0.9982 - val_loss: 0.1108 - val_binary_accuracy: 0.9839 - val_recall_6: 0.9064 - v

In [78]:
model.evaluate(val_batch)

18/18 [==============================] - 0s 9ms/step - loss: 0.1627 - binary_accuracy: 0.9803 - recall_6: 0.8889 - precision_6: 0.9806


[0.16267839074134827,
 0.9802690744400024,
 0.8888888955116272,
 0.9806451797485352]

Even with a very basic model (8,000 embedding params [1000 vocab, 8 dim] and 81 layer params) we get very satisfying results.

Reminder:  
accuracy TP + TN / TP+TN+FP+FN (% of correct predictions)  
recall TP / TP + FN (% of positive values that were correctly predicted)  
precision TP / TP + FP (% of positive predictions that were correct)

We can detect 88% of all spam cases, and 98% of our predictions are correct.

Let's try seeing if using transfer learning of a more complex model can improve our performance.